# MLOps Zoomcamp week-1

In [1]:
import pandas as pd

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet('/home/ubuntu/notebooks/data/fhv_tripdata_2021-01.parquet')

In [3]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


## Q1.

In [4]:
df.shape

(1154112, 7)

### No of records = 1154112

## Q2.

In [5]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime

df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
df.duration.mean()

19.1672240937939

### The average trip duration in January is 19.16 minutes

## No of Records dropped

In [7]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [8]:
df.shape

(1109826, 8)

### No of records dropped = (1154112 - 1109826) = 44286

## Q3.

In [9]:
df.PUlocationID.fillna(-1, inplace=True)
df.DOlocationID.fillna(-1, inplace=True)

In [10]:
(df.PUlocationID[df.PUlocationID == -1].shape[0])/df.shape[0] * 100

83.52732770722618

### The fractions of missing values for the pickup location ID is 83.52%

## Q4.

In [11]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].astype(str)

In [13]:
columns = categorical + ['duration']

In [12]:
train_dicts = df[categorical].to_dict(orient='records')

In [33]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [34]:
X_train.shape[1]

525

### The dimensionality of this matrix is 525

## Q5.

In [17]:
df = df[columns]

In [18]:
y_train = df['duration'].values

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.5285191072072

### The RSME on train dataset is 10.52

## Q6.

In [35]:
val_df = pd.read_parquet('/home/ubuntu/notebooks/data/fhv_tripdata_2021-02.parquet')

In [36]:
val_df['duration'] = val_df.dropOff_datetime - val_df.pickup_datetime

val_df.duration = val_df.duration.apply(lambda td: td.total_seconds() / 60)

val_df = val_df[(val_df.duration >= 1) & (val_df.duration <= 60)]

val_df = val_df[columns]

val_df = val_df.fillna(-1)

val_df[categorical] = val_df[categorical].astype(str)

In [37]:
val_dicts = val_df[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
X_val.shape

(990113, 525)

In [38]:
y_val = val_df.duration.values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.014283163400654

###  The RMSE on validation dataset is 11.01